<a href="https://colab.research.google.com/github/Kenny625819/Applied-Data-Science/blob/main/ESCC_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Unified Reproducible Analysis Script (Submission-Ready)
Multimodal Survival Prediction in Metastatic Spine Surgery

This script reproduces:
    • Leakage-free OOF LightGBM models with isotonic calibration
    • ROC + Calibration plots (3M, 6M, 12M)
    • SHAP summary plots
    • SHAP heatmap with finalized feature labels
    • Performance_Summary_OOF.xlsx
        - AUC, 95% CI, Sensitivity, Specificity, F1, Brier (AI)
        - AUC, Sensitivity, Specificity, F1 (Tokuhashi, Katagiri)
    • Ablation_Study_Output.xlsx
        - Full model (clinical + ESCC) vs Clinical-only
        - AUC, Brier, Sensitivity, Specificity, F1, ΔAUC, DeLong p

Input:
    MRI_ALLdata_OOF.xlsx

Outputs:
    ./RESULTS_FIGURES/
"""

# Optional for Colab:
# !pip install lightgbm shap scikit-learn pandas matplotlib seaborn openpyxl xlsxwriter -q

import numpy as np
import pandas as pd
import lightgbm as lgb
import shap
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from matplotlib.colors import LinearSegmentedColormap

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    roc_curve, roc_auc_score, brier_score_loss,
    confusion_matrix, precision_recall_fscore_support, f1_score
)
from sklearn.isotonic import IsotonicRegression
from scipy.stats import norm


# =============================================================================
# 1. Global settings
# =============================================================================
plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["axes.unicode_minus"] = False

BLUE   = "#0072B2"
ORANGE = "#E69F00"
GREEN  = "#009E73"

OUT_DIR = Path("./RESULTS_FIGURES")
OUT_DIR.mkdir(exist_ok=True)


# =============================================================================
# 2. Preprocessing
# =============================================================================
def map_sex(x):
    s = str(x).strip().lower()
    if s in ["1", "m", "male", "男", "男性"]:
        return 1
    if s in ["0", "f", "female", "女", "女性"]:
        return 0
    return np.nan


def map_escc(x):
    s = str(x).strip().lower().replace(" ", "")
    return {"1b": 2, "1c": 3, "2": 4, "3": 5}.get(s, np.nan)


def frankel_bin(x):
    s = str(x).strip().upper()
    if s in ["A", "B", "C"]:
        return 0
    if s in ["D", "E"]:
        return 1
    return np.nan


def map_yesno(x):
    s = str(x).strip().lower()
    if s in ["yes", "y", "true", "1", "あり", "有"]:
        return 1
    if s in ["no", "n", "false", "0", "なし", "無"]:
        return 0
    return np.nan


df_raw = pd.read_excel("MRI_ALLdata_OOF.xlsx")
df_raw.columns = df_raw.columns.str.strip()

df = pd.DataFrame({
    "Age": df_raw["Age"],
    "Sex": df_raw["Sex"].apply(map_sex),
    "Number of spinal metastases": pd.to_numeric(
        df_raw["Number of Spinal Metastases"], errors="coerce"
    ),
    "Albumin": pd.to_numeric(df_raw["Serum Albumin"], errors="coerce"),
    "CRP": pd.to_numeric(df_raw["CRP"], errors="coerce"),
    "ESCC": df_raw["ESCC"].apply(map_escc),
    "ECOG": pd.to_numeric(df_raw["Performance Status (ECOG)"], errors="coerce"),
    "Frankel grade": df_raw["Frankel Grade"].apply(frankel_bin),
    "Barthel Index": pd.to_numeric(df_raw["Barthel Index (ADL)"], errors="coerce"),
    "Malignancy (New Katagiri score)": pd.to_numeric(
        df_raw["Malignancy (Katagiri Score)"], errors="coerce"
    ),
    "Visceral metastasis": df_raw["Visceral Metastasis (Yes=1/No=0)"].apply(map_yesno),
    "BMI": pd.to_numeric(df_raw["Body Mass Index (BMI)"], errors="coerce"),

    "Tokuhashi_binary": (
        pd.to_numeric(df_raw["Revised Tokuhashi score"], errors="coerce") >= 9
    ).astype(int),
    "Katagiri_binary": (
        pd.to_numeric(df_raw["New Katagiri score"], errors="coerce") < 7
    ).astype(int),

    "Y_3M": pd.to_numeric(df_raw["3-Month Survival (0=Death, 1=Alive)"], errors="coerce"),
    "Y_6M": pd.to_numeric(df_raw["6-Month Survival (0=Death, 1=Alive)"], errors="coerce"),
    "Y_12M": pd.to_numeric(df_raw["12-Month Survival (0=Death, 1=Alive)"], errors="coerce"),
})


# =============================================================================
# 3. Features & SHAP labels (final決定版)
# =============================================================================
FEATURES = [
    "CRP",
    "Malignancy (New Katagiri score)",
    "Age",
    "Barthel Index",
    "Albumin",
    "BMI",
    "ESCC",
    "ECOG",
    "Number of spinal metastases",
    "Sex",
    "Frankel grade",
    "Visceral metastasis",
]

FEATURE_RENAME = {
    "CRP": "CRP",
    "Malignancy (New Katagiri score)": "Malignancy (New Katagiri score)",
    "Age": "Age",
    "Barthel Index": "Barthel Index",
    "Albumin": "Albumin",
    "BMI": "BMI",
    "ESCC": "ESCC grade",
    "ECOG": "ECOG performance status",
    "Number of spinal metastases": "Number of spinal metastases",
    "Sex": "Sex",
    "Frankel grade": "Frankel grade",
    "Visceral metastasis": "Visceral metastasis",
}

FEATURES_NO_ESCC = [f for f in FEATURES if f != "ESCC"]


# =============================================================================
# 4. Metrics
# =============================================================================
def auc_ci_bootstrap(y, p, n_boot=2000):
    y = np.asarray(y)
    p = np.asarray(p)
    rng = np.random.default_rng(42)
    idx = np.arange(len(y))
    aucs = []
    for _ in range(n_boot):
        b = rng.choice(idx, len(idx), replace=True)
        try:
            aucs.append(roc_auc_score(y[b], p[b]))
        except ValueError:
            pass
    auc = roc_auc_score(y, p)
    lo, hi = np.percentile(aucs, [2.5, 97.5])
    return auc, lo, hi


def delong_test(y, p1, p2):
    y = np.asarray(y)
    p1 = np.asarray(p1)
    p2 = np.asarray(p2)
    order = np.argsort(-p1)
    y, p1, p2 = y[order], p1[order], p2[order]
    m = int(np.sum(y))
    n = len(y) - m
    if m == 0 or n == 0:
        return np.nan, np.nan

    def auc_stat(p):
        r = pd.Series(p).rank().values
        return (np.sum(r[:m]) - m * (m + 1) / 2) / (m * n)

    a1 = auc_stat(p1)
    a2 = auc_stat(p2)
    var = (a1 * (1 - a1) + a2 * (1 - a2)) / min(m, n)
    z = (a1 - a2) / np.sqrt(var + 1e-12)
    p = 2 * (1 - norm.cdf(abs(z)))
    return a1 - a2, p


def get_binary_metrics(y, p, thr=0.5):
    y = np.asarray(y)
    p = np.asarray(p)
    yhat = (p >= thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y, yhat).ravel()
    sens = tp / (tp + fn) if (tp + fn) > 0 else np.nan
    spec = tn / (tn + fp) if (tn + fp) > 0 else np.nan
    _, _, f1, _ = precision_recall_fscore_support(
        y, yhat, average="binary", zero_division=0
    )
    return sens, spec, f1


def youden_threshold(y, p):
    fpr, tpr, thr = roc_curve(y, p)
    return thr[np.argmax(tpr - fpr)]


# =============================================================================
# 5. OOF LightGBM + Isotonic calibration
# =============================================================================
def run_lgb_oof_calibrated(X, y):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    params = dict(
        objective="binary",
        metric="auc",
        learning_rate=0.05,
        num_leaves=31,
        n_estimators=500,
        class_weight="balanced",
        random_state=42,
    )
    oof = np.zeros(len(y))

    for tr, te in skf.split(X, y):
        model = lgb.LGBMClassifier(**params)
        model.fit(X.iloc[tr], y[tr])
        oof[te] = model.predict_proba(X.iloc[te])[:, 1]

    iso = IsotonicRegression(out_of_bounds="clip")
    iso.fit(oof, y)
    calibrated = iso.transform(oof)

    final_model = lgb.LGBMClassifier(**params)
    final_model.fit(X, y)

    return calibrated, final_model


# =============================================================================
# 6. Plotting
# =============================================================================
def plot_roc(ax, y, ai, tok, kat, title):
    fpr_ai, tpr_ai, _ = roc_curve(y, ai)
    fpr_t,  tpr_t,  _ = roc_curve(y, tok)
    fpr_k,  tpr_k,  _ = roc_curve(y, kat)

    ax.plot(
        fpr_ai, tpr_ai,
        color=BLUE, lw=2.5,
        label=f"AI model (AUC={roc_auc_score(y, ai):.3f})",
    )
    ax.plot(
        fpr_t, tpr_t,
        color=ORANGE, ls="--", lw=2,
        label=f"Revised Tokuhashi score (AUC={roc_auc_score(y, tok):.3f})",
    )
    ax.plot(
        fpr_k, tpr_k,
        color=GREEN, ls=":", lw=2,
        label=f"New Katagiri score (AUC={roc_auc_score(y, kat):.3f})",
    )
    ax.plot([0, 1], [0, 1], "--", color="gray")
    ax.set_title(title, fontsize=20)
    ax.set_xlabel("1 − Specificity", fontsize=20)
    ax.set_ylabel("Sensitivity", fontsize=20)
    ax.legend(loc="lower right", fontsize=12)
    ax.tick_params(labelsize=20)


def plot_calibration(ax, y, p):
    d = pd.DataFrame({"y": y, "p": p})
    d["bin"] = pd.qcut(d["p"], q=10, duplicates="drop")
    g = d.groupby("bin").agg(obs=("y", "mean"), pred=("p", "mean")).reset_index()
    ax.plot([0, 1], [0, 1], "--", color="gray")
    ax.plot(g["pred"], g["obs"], "o-", color=BLUE)
    ax.set_title("Calibration plot", fontsize=20)
    ax.set_xlabel("Predicted probability", fontsize=20)
    ax.set_ylabel("Observed frequency", fontsize=20)
    ax.tick_params(labelsize=20)


def shap_summary_plot(model, X, title, filename):
    X_disp = X.rename(columns=FEATURE_RENAME)
    explainer = shap.TreeExplainer(model)
    sv = explainer.shap_values(X)
    if isinstance(sv, list):
        sv = sv[1]
    shap.summary_plot(sv, X_disp, show=False, plot_size=(8, 6))
    plt.title(title, fontsize=18)
    plt.tight_layout()
    plt.savefig(OUT_DIR / filename, dpi=600)
    plt.close()


def shap_mean_heatmap(models, X_dict, features, filename):
    cmap = LinearSegmentedColormap.from_list("roc_blue", ["#e6f2ff", "#1f77b4"])
    disp = [FEATURE_RENAME[f] for f in features]
    table = pd.DataFrame(index=disp, columns=models.keys())

    for tag, model in models.items():
        X = X_dict[tag]
        explainer = shap.TreeExplainer(model)
        sv = explainer.shap_values(X)
        if isinstance(sv, list):
            sv = sv[1]
        abs_mean = np.abs(sv).mean(axis=0)
        for orig, lab, val in zip(features, disp, abs_mean):
            table.loc[lab, tag] = val

    if "3M" in models:
        table = table.sort_values(by="3M", ascending=False)

    plt.figure(figsize=(8, 7))
    sns.heatmap(
        table.astype(float),
        cmap=cmap,
        annot=True,
        fmt=".3f",
        annot_kws={"fontsize": 11},
        cbar_kws={"label": "mean(|SHAP value|)"},
    )
    plt.title("SHAP Heatmap", fontsize=18)
    plt.xticks(fontsize=13)
    plt.yticks(fontsize=13)
    plt.tight_layout()
    plt.savefig(OUT_DIR / filename, dpi=600)
    plt.close()


# =============================================================================
# 7. Main survival analysis (3M / 6M / 12M)
# =============================================================================
perf_records = []
models_by_horizon = {}
X_dict = {}

for tag, ycol in [("3M", "Y_3M"), ("6M", "Y_6M"), ("12M", "Y_12M")]:

    sub = df.dropna(subset=FEATURES + [ycol])
    X = sub[FEATURES]
    y = sub[ycol].astype(int).values

    X_dict[tag] = X
    p_ai, model_ai = run_lgb_oof_calibrated(X, y)
    models_by_horizon[tag] = model_ai

    p_tok = sub["Tokuhashi_binary"].astype(float).values
    p_kat = sub["Katagiri_binary"].astype(float).values

    # Plots
    fig, axes = plt.subplots(1, 2, figsize=(13, 5))
    plot_roc(axes[0], y, p_ai, p_tok, p_kat, f"{tag} survival")
    plot_calibration(axes[1], y, p_ai)
    plt.tight_layout()
    plt.savefig(OUT_DIR / f"Figure3_{tag}.png", dpi=600)
    plt.close()

    shap_summary_plot(model_ai, X, f"SHAP summary ({tag})", f"SHAP_{tag}.png")

    # Metrics
    ai_auc, ai_lo, ai_hi = auc_ci_bootstrap(y, p_ai)
    tk_auc, tk_lo, tk_hi = auc_ci_bootstrap(y, p_tok)
    kt_auc, kt_lo, kt_hi = auc_ci_bootstrap(y, p_kat)

    _, p_ai_vs_tok = delong_test(y, p_ai, p_tok)
    _, p_ai_vs_kat = delong_test(y, p_ai, p_kat)

    thr_ai = youden_threshold(y, p_ai)
    sens_ai, spec_ai, f1_ai = get_binary_metrics(y, p_ai, thr_ai)
    sens_t, spec_t, f1_t = get_binary_metrics(y, p_tok, 0.5)
    sens_k, spec_k, f1_k = get_binary_metrics(y, p_kat, 0.5)

    brier_ai = brier_score_loss(y, p_ai)

    perf_records.append({
        "Horizon": tag,

        "AI AUC": ai_auc,
        "AI 95% CI low": ai_lo,
        "AI 95% CI high": ai_hi,
        "AI Sensitivity": sens_ai,
        "AI Specificity": spec_ai,
        "AI F1": f1_ai,
        "AI Brier": brier_ai,

        "Tokuhashi AUC": tk_auc,
        "Tokuhashi 95% CI low": tk_lo,
        "Tokuhashi 95% CI high": tk_hi,
        "Tokuhashi Sensitivity": sens_t,
        "Tokuhashi Specificity": spec_t,
        "Tokuhashi F1": f1_t,

        "Katagiri AUC": kt_auc,
        "Katagiri 95% CI low": kt_lo,
        "Katagiri 95% CI high": kt_hi,
        "Katagiri Sensitivity": sens_k,
        "Katagiri Specificity": spec_k,
        "Katagiri F1": f1_k,

        "p (AI vs Tokuhashi)": p_ai_vs_tok,
        "p (AI vs Katagiri)": p_ai_vs_kat,

        "n": len(sub),
    })

perf_df = pd.DataFrame(perf_records)
perf_df.to_excel(OUT_DIR / "Performance_Summary_OOF.xlsx", index=False)

shap_mean_heatmap(
    models_by_horizon, X_dict, FEATURES, "SHAP_heatmap_3M_6M_12M.png"
)


# =============================================================================
# 8. Ablation Study (Full vs Clinical-only)
# =============================================================================
ablation_records = []

for tag, ycol in [("3M", "Y_3M"), ("6M", "Y_6M"), ("12M", "Y_12M")]:

    sub = df.dropna(subset=FEATURES + [ycol])
    X_full = sub[FEATURES]
    X_no   = sub[FEATURES_NO_ESCC]
    y = sub[ycol].astype(int).values

    p_full, _ = run_lgb_oof_calibrated(X_full, y)
    p_no, _   = run_lgb_oof_calibrated(X_no, y)

    auc_full = roc_auc_score(y, p_full)
    auc_no   = roc_auc_score(y, p_no)

    brier_full = brier_score_loss(y, p_full)
    brier_no   = brier_score_loss(y, p_no)

    sens_full, spec_full, f1_full = get_binary_metrics(y, p_full, 0.5)
    sens_no,   spec_no,   f1_no   = get_binary_metrics(y, p_no,   0.5)

    delta_auc, p_delong = delong_test(y, p_full, p_no)

    ablation_records.append({
        "Horizon": tag,
        "AUC_Full (clinical + ESCC)": auc_full,
        "AUC_No_ESCC (clinical only)": auc_no,
        "ΔAUC (Full − No_ESCC)": delta_auc,
        "p (DeLong)": p_delong,
        "Brier_Full": brier_full,
        "Brier_No_ESCC": brier_no,
        "Sensitivity_Full": sens_full,
        "Specificity_Full": spec_full,
        "F1_Full": f1_full,
        "Sensitivity_No_ESCC": sens_no,
        "Specificity_No_ESCC": spec_no,
        "F1_No_ESCC": f1_no,
        "n": len(sub),
    })

ablation_df = pd.DataFrame(ablation_records)
ablation_df.to_excel(OUT_DIR / "Ablation_Study_Output.xlsx", index=False)

print("Analysis completed successfully.")
print("Results saved in:", OUT_DIR.resolve())


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 110, number of negative: 30
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000165 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 182
[LightGBM] [Info] Number of data points in the train set: 140, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

/tmp/ipython-input-1364890607.py:291: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = d.groupby("bin").agg(obs=("y", "mean"), pred=("p", "mean")).reset_index()
/usr/local/lib/python3.12/dist-packages/shap/explainers/_tree.py:587: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 84, number of negative: 56
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000048 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 178
[LightGBM] [Info] Number of data points in the train set: 140, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

/tmp/ipython-input-1364890607.py:291: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = d.groupby("bin").agg(obs=("y", "mean"), pred=("p", "mean")).reset_index()
/usr/local/lib/python3.12/dist-packages/shap/explainers/_tree.py:587: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 62, number of negative: 78
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000049 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 181
[LightGBM] [Info] Number of data points in the train set: 140, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

/tmp/ipython-input-1364890607.py:291: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = d.groupby("bin").agg(obs=("y", "mean"), pred=("p", "mean")).reset_index()
/usr/local/lib/python3.12/dist-packages/shap/explainers/_tree.py:587: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/shap/explainers/_tree.py:587: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/shap/explainers/_tree.py:587: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/shap/explain

ストリーミング出力は最後の 5000 行に切り捨てられました。
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain